## 04 Model Training

#### Import relevant libraries

In [1]:
import os
import sklearn as skl
import pandas as pd
import numpy as np

#### Load model training data

In [2]:
input_dir = os.path.join('..', 'data', 'processed')
df = pd.read_csv(os.path.join(input_dir, 'model_data.csv'))

In [3]:
df.head()

,target,x01,x02,x03,x04,x05,x06,x07,x08,c09
0,0.019042,0.351208,0.110678,95.893125,1.000000,0.290916,0.778672,0.220809,0.000000,vert_int
1,0.015868,0.513455,0.476233,31.004672,1.000000,0.916615,0.000000,1.000000,0.000000,vert_int
2,0.022832,0.642341,0.039913,54.876484,1.000005,0.024628,0.000000,1.000080,10.487586,vert_int
3,0.012631,0.419923,0.251899,77.037977,1.000554,0.325085,0.963355,0.000000,14.585460,vert_int
4,0.019896,0.610624,0.525933,65.810615,1.000000,0.537077,0.990224,0.009776,0.000000,vert_int


#### One-hot encoding categorical variables

In [4]:
categorical_vars = df.columns[[col_name.startswith('c') for col_name in df.columns]]
c_vars = pd.get_dummies(df[categorical_vars], drop_first=True)
c_vars.head()

,c09_other_wires,c09_restructured,c09_vert_int
0,0,0,1
1,0,0,1
2,0,0,1
3,0,0,1
4,0,0,1


#### Scale predictor variables

In [5]:
from sklearn.preprocessing import scale
continuous_vars = df.columns[[col_name.startswith('x') for col_name in df.columns]]
x_vars = df[continuous_vars].copy()

for col_name in continuous_vars:
    x_vars[col_name] = scale(df[col_name])

x_vars.head()

,x01,x02,x03,x04,x05,x06,x07,x08
0,-1.208260,-0.283548,0.592913,-0.178322,0.449758,1.710097,0.053862,-0.710241
1,-0.490188,1.377255,-1.021040,-0.178322,2.534275,-0.481968,2.202111,-0.710241
2,0.080234,-0.605050,-0.427283,-0.178248,-0.437378,-0.481968,2.202332,0.048291
3,-0.904142,0.358051,0.123934,-0.170482,0.563595,2.230005,-0.554915,0.344676
4,-0.060139,1.603053,-0.155321,-0.178322,1.269845,2.305643,-0.527961,-0.710241


#### Split into test and train sets

In [6]:
y = df['target']
X = x_vars.join(c_vars)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)
print(X_train.shape, y_train.shape); print(X_test.shape, y_test.shape);

(92, 11) (92,)
(24, 11) (24,)


Save as csv for use in fit evaluation step

In [8]:
X_train.to_csv(os.path.join(input_dir, 'X_train.csv'), index=False)
X_test.to_csv(os.path.join(input_dir, 'X_test.csv'), index=False)
y_train.to_csv(os.path.join(input_dir, 'y_train.csv'), index=False)
y_test.to_csv(os.path.join(input_dir, 'y_test.csv'), index=False)

In [9]:
np.save(os.path.join(input_dir, 'X_train'), X_train)
np.save(os.path.join(input_dir, 'X_test'), X_test)
np.save(os.path.join(input_dir, 'y_train'), y_train)
np.save(os.path.join(input_dir, 'y_test'), y_test)

#### Fit regression model

(a) Linear regression

In [10]:
# Import LinearRegression from sklearn
from sklearn.linear_model import LinearRegression

# Instantiate the regressor
reg_lin = LinearRegression()

# Fit the regressor to the training data
reg_lin.fit(X_train, y_train)

# Test the linear regression model
predicted = reg_lin.predict(X_test)
print(reg_lin.intercept_, reg_lin.coef_)

0.04136570420241568 [-0.0160827  -0.00161387 -0.00104436 -0.00302767  0.00448619 -0.00021513
 -0.00020368 -0.0041874  -0.0255398  -0.00886454 -0.03421114]


(b) Random forest

In [11]:
# Import RandomForestRegressor from sklearn
from sklearn.ensemble import RandomForestRegressor

# Instantiate the regressor with default value for hyperparameters
reg_forest = RandomForestRegressor(random_state=123)

# Fit the regressor to the training data
reg_forest.fit(X_train, y_train)
reg_forest.feature_importances_

array([0.33819844, 0.02471854, 0.29471404, 0.1622484 , 0.05630196,
       0.01280448, 0.02497056, 0.0439977 , 0.02562598, 0.00839059,
       0.0080293 ])

(c) Neural Network

In [12]:
# Import MLPRegressor from skleatn
from sklearn.neural_network import MLPRegressor

#Instantiate the regressor with default values
reg_neural = MLPRegressor(random_state=123)

# Fit the regressor to the training data
reg_neural.fit(X_train, y_train)

MLPRegressor(random_state=123)

#### Dump model_fit file(s)

In [13]:
# Import the dump function
from joblib import dump

In [14]:
# Save linear regression model to disk
filename = os.path.join(input_dir,'lin_model_3.joblib')
dump(reg_lin, open(filename, 'wb'))

In [15]:
# Save random forest model to disk
filename = os.path.join(input_dir,'forest_model_1.joblib')
dump(reg_forest, open(filename, 'wb'))

In [16]:
# Save neural network model to disk
filename = os.path.join(input_dir,'neural_model_1.joblib')
dump(reg_neural, open(filename, 'wb'))